# Offload and Idle Pull

This notebook demonstrates the workload behaviour for big core offload and idle pull

In [1]:
import logging
reload(logging)
logging.basicConfig(
    format='%(asctime)-9s %(levelname)-8s: %(message)s',
    datefmt='%I:%M:%S')

# Enable logging at INFO level
logging.getLogger().setLevel(logging.INFO)
# Comment the follwing line to disable devlib debugging statements
# logging.getLogger('ssh').setLevel(logging.DEBUG)

In [2]:
# Generate plots inline
%pylab inline

import json
import os
import subprocess

# Support to access the remote target
import devlib
from env import TestEnv

# Support for trace events analysis
from trace import Trace
from trace_analysis import TraceAnalysis

# Support to configure and run RTApp based workloads
from wlgen import RTA

# Support for performance analysis of RTApp workloads
from perf_analysis import PerfAnalysis

# Suport for FTrace events parsing and visualization
import trappy

Populating the interactive namespace from numpy and matplotlib


# Test environment setup

In [3]:
# Setup a target configuration
my_target_conf = {
    
    # Define the kind of target platform to use for the experiments
    "platform"    : 'android',  # Linux system, valid other options are:
                              # android - access via ADB
                              # linux   - access via SSH
                              # host    - direct access
    
    # Preload settings for a specific target
    "board"       : 'juno',   # load JUNO specific settings, e.g.
                              # - HWMON based energy sampling
                              # - Juno energy model
                              # valid options are:
                              # - juno  - JUNO Development Board
                              # - tc2   - TC2 Development Board
                              # - oak   - Mediatek MT63xx based target

    # Define devlib module to load
    #"modules"     : [
    #    'bl',           # enable big.LITTLE support
    #    'cpufreq'       # enable CPUFreq support
    #],

    # Account to access the remote target
    "host"        : '10.169.36.25',
    "username"    : 'root',
    "password"    : '',

    # Comment the following line to force rt-app calibration on your target
    "rtapp-calib" : {
        '0': 361, '1': 138, '2': 138, '3': 352, '4': 360, '5': 353
    }

}

# Setup the required Test Environment supports
my_tests_conf = {
    
    # Binary tools required to run this experiment
    # These tools must be present in the tools/ folder for the architecture
    "tools"   : ['rt-app', 'taskset', 'trace-cmd'],
    
    # FTrace events end buffer configuration
    "ftrace"  : {
         "events" : [
            "cpu_frequency",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_switch",
            'dequeue_task_fair', 
            'enqueue_task_fair', 
            'set_next_entity',

                    
            "cpu_capacity",
            
            #/sys/kernel/debug/tracing/events/sched
            "sched_blocked_reason",
            "sched_boost_cpu",
            "sched_boost_task",
            "sched_contrib_scale_f",
            "sched_cpu_hotplug",
            "sched_energy_diff",
            "sched_kthread_stop",
            "sched_kthread_stop_ret",
            "sched_load_avg_cpu",
            "sched_load_avg_task",
            "sched_migrate_task",
            "sched_move_numa",
            "sched_pi_setprio",
            "sched_process_exec",
            "sched_process_exit",
            "sched_process_fork",
            "sched_process_free",
            "sched_process_wait",
            "sched_stat_blocked",
            "sched_stat_iowait",
            "sched_stat_runtime",
            "sched_stat_sleep",
            "sched_stat_wait",
            "sched_stick_numa",
            "sched_swap_numa",
            "sched_switch",
            "sched_tune_boostgroup_update",
            "sched_tune_config",
            "sched_tune_tasks_update",
            "sched_tune_filter",
            "sched_wait_task",
            "sched_wake_idle_without_ipi",
            "sched_wakeup",
            "sched_wakeup_new",
            
            
         ],
         "buffsize" : 10240
    },
    "results_dir" : "lisa_offload_and_idle_pull",
}

In [5]:
# Support to access the remote target
import devlib
from env import TestEnv

# Initialize a test environment using:
# the provided target configuration (my_target_conf)
# the provided test configuration   (my_test_conf)
te = TestEnv(target_conf=my_target_conf, test_conf=my_tests_conf)
target = te.target

02:22:52  INFO    :         Target - Using base path: /data/lisa_fork/lisa
02:22:52  INFO    :         Target - Loading custom (inline) target configuration
02:22:52  INFO    :         Target - Loading custom (inline) test configuration
02:22:52  INFO    :         Target - External tools using:
02:22:52  INFO    :         Target -    ANDROID_HOME: /home/zhifei/zyang/android-sdk-linux
02:22:52  INFO    :         Target -    CATAPULT_HOME: /data/lisa_fork/lisa/tools/catapult
02:22:52  INFO    :         Target - Devlib modules to load: ['bl', 'hwmon', 'cpufreq']
02:22:52  INFO    :         Target - Connecting Android target [10.169.36.25:5555]
02:22:52  INFO    :         Target - Connection settings:
02:22:52  INFO    :         Target -    {'device': '10.169.36.25:5555'}
02:22:55  INFO    :         Target - Initializing target workdir:
02:22:55  INFO    :         Target -    /data/local/tmp/devlib-target
02:23:02  INFO    :         Target - Topology:
02:23:02  INFO    :         Target -  

# Workload configuration

In [6]:
# Support to configure and run RTApp based workloads
from wlgen import RTA, Periodic, Ramp

# Create a new RTApp workload generator using the calibration values
# reported by the TestEnv module
rtapp = RTA(target, 'simple', calibration=te.calibration())

# Configure this RTApp instance to:
rtapp.conf(
    # 1. generate a "profile based" set of tasks
    kind='profile',
    
    # 2. define the "profile" of each task
    params={
        
        "early_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
        ).get(),

        "early_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
        ).get(),

        
        "delay_starter_1" : Periodic(
            period_ms=10,
            duty_cycle_pct=100,
            duration_s=5,
            delay_s=1,
        ).get(),
        
        "delay_starter_2" : Periodic(
            period_ms=10,
            duty_cycle_pct=5,
            duration_s=10,
            delay_s=1,
        ).get(),

#        "delay_starter_3" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
#        
#        "delay_starter_4" : Periodic(
#            period_ms=10,
#            duty_cycle_pct=100,
#            duration_s=5,
#            delay_s=2,
#        ).get(),
    },
    
    # 4. use this folder for task logfiles
    run_dir=target.working_directory
    
);

02:23:19  INFO    :          WlGen - Setup new workload simple
02:23:19  DEBUG   :          WlGen - Setup step [postrun] callback to [__postrun] function
02:23:19  DEBUG   :          WlGen - Configuring a profile-based workload...
02:23:19  DEBUG   :          RTApp - ref on big cpu: 1
02:23:19  INFO    :          RTApp - Workload duration defined by longest task
02:23:19  INFO    :          RTApp - Default policy: SCHED_OTHER
02:23:19  INFO    :          RTApp - ------------------------
02:23:19  INFO    :          RTApp - task [delay_starter_1], sched: using default policy
02:23:19  INFO    :          RTApp -  | start delay: 1.000000 [s]
02:23:19  INFO    :          RTApp -  | calibration CPU: 1
02:23:19  INFO    :          RTApp -  | loops count: 1
02:23:19  INFO    :          RTApp -  + phase_000001: batch 5.000000 [s]
02:23:19  INFO    :          RTApp - ------------------------
02:23:19  INFO    :          RTApp - task [delay_starter_2], sched: using default policy
02:23:19  INFO 

# Workload execution

In [7]:
# Set Sched governor
#
# For Juno big/LITTLE CPUs
#
logging.info("Target ABI: %s, CPus: %s",
             target.abi,
             target.cpuinfo.cpu_names)

target.cpufreq.set_all_governors('sched')

logging.info("Target current governor: %s",
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor")
            )
logging.info("Target big CPU max CPUfreq:%s\n\t\t    Target big CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq")
             )

logging.info("Target LITTLE CPU max CPUfreq:%s \n\t\t    Target LITTLE CPU current CPUfreq: %s", 
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq"),
             target.read_value("/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq")
             )

02:23:30  INFO    : Target ABI: arm64, CPus: ['A53', 'A57', 'A57', 'A53', 'A53', 'A53']
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "echo '/data/local/tmp/bin/shutils cpufreq_set_all_governors sched' | su"
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_governor'"
02:23:30  INFO    : Target current governor: sched
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_max_freq'"
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/devices/system/cpu/cpu0/cpufreq/scaling_cur_freq'"
02:23:30  INFO    : Target big CPU max CPUfreq:850000
		    Target big CPU current CPUfreq: 850000
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_max_freq'"
02:23:30  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/devices/system/cpu/cpu1/cpufreq/scaling_cur_freq'"
02:23:30  INFO    : Target LITTLE CPU max CPUfreq:1100000 
		    Target 

In [ ]:
logging.info('#### Setup FTrace')
te.ftrace.start()

logging.info('#### Start energy sampling')
te.emeter.reset()

logging.info('#### Start RTApp execution')
rtapp.run(out_dir=te.res_dir, cgroup="")

logging.info('#### Read energy consumption: %s/energy.json', te.res_dir)
(nrg, nrg_file) = te.emeter.report(out_dir=te.res_dir)

logging.info('#### Stop FTrace')
te.ftrace.stop()

trace_file = os.path.join(te.res_dir, 'trace.dat')
logging.info('#### Save FTrace: %s', trace_file)
te.ftrace.get_trace(trace_file)

logging.info('#### Save platform description: %s/platform.json', te.res_dir)
(plt, plt_file) = te.platform_dump(te.res_dir)

02:23:32  INFO    : #### Setup FTrace
02:23:32  DEBUG   : adb -s 10.169.36.25:5555 shell "echo 'echo 10240 > '\\''/sys/kernel/debug/tracing/buffer_size_kb'\\''' | su"
02:23:32  DEBUG   : adb -s 10.169.36.25:5555 shell "cat '/sys/kernel/debug/tracing/buffer_size_kb'"
02:23:32  DEBUG   : adb -s 10.169.36.25:5555 shell "echo '/data/local/tmp/bin/trace-cmd reset' | su"
02:23:33  DEBUG   : adb -s 10.169.36.25:5555 shell "echo '/data/local/tmp/bin/trace-cmd start -e cpu_frequency -e sched_load_avg_cpu -e sched_load_avg_task -e sched_switch -e cpu_capacity -e sched_blocked_reason -e sched_boost_cpu -e sched_boost_task -e sched_contrib_scale_f -e sched_cpu_hotplug -e sched_energy_diff -e sched_kthread_stop -e sched_kthread_stop_ret -e sched_load_avg_cpu -e sched_load_avg_task -e sched_migrate_task -e sched_move_numa -e sched_pi_setprio -e sched_process_exec -e sched_process_exit -e sched_process_fork -e sched_process_free -e sched_process_wait -e sched_stat_blocked -e sched_stat_iowait -e sche

# Collected results

In [28]:
# All data are produced in the output folder defined by the TestEnv module
logging.info('Content of the output folder %s', te.res_dir)
!ls -la {te.res_dir}

05:25:21  INFO    : Content of the output folder /data/lisa_fork/lisa/results/lisa_offload_and_idle_pull


total 27768
drwxrwxr-x  2 zhifei zhifei     4096 2月  22 17:25 .
drwxrwxr-x 22 zhifei zhifei     4096 2月  22 17:15 ..
-rw-rw-r--  1 zhifei zhifei       68 2月  22 17:25 energy.json
-rw-rw-r--  1 zhifei zhifei      581 2月  22 17:25 output.log
-rw-rw-r--  1 zhifei zhifei     1075 2月  22 17:25 platform.json
-rw-r--r--  1 zhifei zhifei      408 2月  22 17:25 rt-app-delay_starter_1-0.log
-rw-r--r--  1 zhifei zhifei   124284 2月  22 17:25 rt-app-delay_starter_2-1.log
-rw-r--r--  1 zhifei zhifei      284 2月  22 17:25 rt-app-early_starter_1-2.log
-rw-r--r--  1 zhifei zhifei      284 2月  22 17:25 rt-app-early_starter_2-3.log
-rw-r--r--  1 zhifei zhifei     1506 2月  22 17:25 simple_00.json
-rw-r--r--  1 zhifei zhifei  7581696 2月  22 17:25 trace.dat
-rw-rw-r--  1 zhifei zhifei 10575633 2月  22 17:21 trace.raw.txt
-rw-rw-r--  1 zhifei zhifei 10110709 2月  22 17:21 trace.txt


In [29]:
# Inspect the JSON file used to run the application
with open('{}/simple_00.json'.format(te.res_dir), 'r') as fh:
    rtapp_json = json.load(fh, )
logging.info('Generated RTApp JSON file:')
#print json.dumps(rtapp_json, indent=4, sort_keys=True)

05:25:21  INFO    : Generated RTApp JSON file:


In [30]:
# Dump the energy measured for the LITTLE and big clusters
logging.info('Energy: %s', nrg_file)
print json.dumps(nrg, indent=4, sort_keys=True)

05:25:21  INFO    : Energy: /data/lisa_fork/lisa/results/lisa_offload_and_idle_pull/energy.json


{
    "LITTLE": 1.1715649999999869, 
    "big": 10.895853999999986
}


In [31]:
# Dump the platform descriptor, which could be useful for further analysis
# of the generated results
logging.info('Platform description: %s', plt_file)
#print json.dumps(plt, indent=4, sort_keys=True)

05:25:21  INFO    : Platform description: /data/lisa_fork/lisa/results/lisa_offload_and_idle_pull/platform.json


# Trace inspection

In [32]:
# Suport for FTrace events parsing and visualization
import trappy

# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(te.res_dir)

# New Task Behavior

In [33]:
events_to_parse = my_tests_conf['ftrace']['events']

trace = Trace(plt, te.res_dir, events_to_parse)

ftrace = trace.ftrace

trappy.plotter.plot_trace(ftrace, execnames=['early_starter_1', "early_starter_2", "delay_starter_1",
                                             "delay_starter_2", "delay_starter_3", "delay_starter_4",
                                            ])

05:25:34  DEBUG   : Loading [sched] events from trace in [/data/lisa_fork/lisa/results/lisa_offload_and_idle_pull]...
05:25:34  DEBUG   : Parsing events: ['cpu_frequency', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_switch', 'dequeue_task_fair', 'enqueue_task_fair', 'set_next_entity', 'cpu_capacity', 'sched_blocked_reason', 'sched_boost_cpu', 'sched_boost_task', 'sched_contrib_scale_f', 'sched_cpu_hotplug', 'sched_energy_diff', 'sched_kthread_stop', 'sched_kthread_stop_ret', 'sched_load_avg_cpu', 'sched_load_avg_task', 'sched_migrate_task', 'sched_move_numa', 'sched_pi_setprio', 'sched_process_exec', 'sched_process_exit', 'sched_process_fork', 'sched_process_free', 'sched_process_wait', 'sched_stat_blocked', 'sched_stat_iowait', 'sched_stat_runtime', 'sched_stat_sleep', 'sched_stat_wait', 'sched_stick_numa', 'sched_swap_numa', 'sched_switch', 'sched_tune_boostgroup_update', 'sched_tune_config', 'sched_tune_tasks_update', 'sched_tune_filter', 'sched_wait_task', 'sched_wake_idle_

# Trace Overview

In [24]:
# NOTE: The interactive trace visualization is available only if you run
#       the workload to generate a new trace-file
trappy.plotter.plot_trace(ftrace)